In [3]:
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, roc_auc_score
from sklearn.svm import SVC
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

import random
import pandas as pd
import numpy as np


In [4]:
data = pd.read_csv('./mecab_word2vec_재무비율.csv')
data.rename(columns={'운전자산총자본비율':'운전자본비율'}, inplace=True)
data.fillna(0, inplace=True)
# data.drop(['회사명', '회계년도', '폐지일자'], axis=1, inplace=True)
input = data.iloc[:,2:8]
target = data.iloc[:,8]

columns = ['Accuracy', 'Roc_auc_score', 'Recall']
비교테이블 = pd.DataFrame(columns = columns)

np.random.seed(42)
data

,회사명,회계년도,운전자본비율,이익잉여금총자산비율,총자산이익율,시장가부채비율,매출액회전율,부도기사비율,부실기업여부
0,네이쳐글로벌,2009,74.492049,-49.803589,-3.371182,673.432622,6.028268,10.810811,1
1,네프로아이티,2009,-0.465154,19.338150,13.884726,157.344363,76.780711,38.235294,1
2,뉴젠아이씨티,2009,27.593677,-1075.333708,-28.342808,136.342262,40.747316,16.494845,1
3,트루아워,2009,52.368738,-19.102365,-25.573719,412.297157,33.436299,46.341463,1
4,디패션,2009,20.920391,-333.294194,-23.158301,170.052742,71.074703,30.000000,1
...,...,...,...,...,...,...,...,...,...
265,지니뮤직,2016,45.063046,-14.623948,4.450491,262.390477,101.096537,4.761905,0
266,대주전자재료,2016,-8.786573,9.737698,2.277756,170.648920,48.084200,0.000000,0
267,크리스에프앤씨,2016,30.558431,46.651963,19.919396,213.287219,133.252743,15.789474,0
268,앤디포스,2016,81.281282,35.275974,18.934118,1071.393920,79.493948,0.000000,0


## XGBClassifier
##### 설명 참고 : https://wooono.tistory.com/97

In [33]:
x_train, x_test, y_train, y_test = train_test_split(input, target, random_state=42, test_size=0.2)

rs = RobustScaler()
x_train = rs.fit_transform(x_train)
x_test = rs.fit_transform(x_test)

xgb = XGBClassifier(random_state=42)

xgb_param_grid = {'n_estimators' : [100, 200],
                'learning_rate' : [0.01, 0.05, 0.1],
                'max_depth' : [3, 5, 7],
                'gamma' : [0, 1, 2]}

xgb_grid = GridSearchCV(xgb, param_grid=xgb_param_grid, scoring='roc_auc', verbose=0, n_jobs=1)
xgb_grid.fit(x_train, y_train)

print(f'best f1 : {xgb_grid.best_score_}')
print('best param : ', xgb_grid.best_params_)

## 참고 : https://cjh34544.tistory.com/m/4
## http://aispiration.com/model/model-python-xgboost-hyper.html

best f1 : 0.9255411255411256
best param :  {'gamma': 2, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}


In [4]:
x_train, x_test, y_train, y_test = train_test_split(input, target, random_state=42, test_size=0.2)

xgb = XGBClassifier(random_state=42)

xgb_param_grid = {'n_estimators' : [100, 200],
                'learning_rate' : [0.01, 0.05, 0.1],
                'max_depth' : [3, 5, 7],
                'gamma' : [0, 1, 2]}

xgb_grid = GridSearchCV(xgb, param_grid=xgb_param_grid, scoring='roc_auc', verbose=0, n_jobs=1)
xgb_grid.fit(x_train, y_train)

print(f'best f1 : {xgb_grid.best_score_}')
print('best param : ', xgb_grid.best_params_)

## 참고 : https://cjh34544.tistory.com/m/4
## http://aispiration.com/model/model-python-xgboost-hyper.html

best f1 : 0.9532664305391577
best param :  {'gamma': 2, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200}


## Logistic

In [5]:
x_train, x_test, y_train, y_test = train_test_split(input, target, random_state=42, test_size=0.2)

rs = RobustScaler()
x_train = rs.fit_transform(x_train)
x_test = rs.fit_transform(x_test)

lr = LogisticRegression(random_state=42)

lr_param_grid = {'C' : [0.001, 0.01, 0.1, 1, 10],
                'penalty' : ['l1', 'l2']}

lr_grid = GridSearchCV(lr, param_grid=lr_param_grid, scoring='roc_auc', verbose=0, n_jobs=1)
lr_grid.fit(x_train, y_train)

print(f'best f1 : {lr_grid.best_score_}')
print('best param : ', lr_grid.best_params_)

# 참고 : https://wikidocs.net/16594


best f1 : 0.9374458874458875
best param :  {'C': 0.1, 'penalty': 'l2'}


c:\Users\dbswo\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
25 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\dbswo\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\dbswo\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py", line 1091, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "c:\Users\dbswo\AppData\Local\Programs\Python\

In [6]:
pred = lr_grid.best_estimator_.predict_proba(x_test)
roc = roc_auc_score(y_test, pred[:,1])
print(roc)

0.8804945054945055


In [3]:
x_train, x_test, y_train, y_test = train_test_split(input, target, random_state=42, test_size=0.2)

lr = LogisticRegression(random_state=42)

lr_param_grid = {'C' : [0.001, 0.01, 0.1, 1, 10],
                'penalty' : ['l1', 'l2']}

lr_grid = GridSearchCV(lr, param_grid=lr_param_grid, scoring='roc_auc', verbose=0, n_jobs=1)
lr_grid.fit(x_train, y_train)

print(f'best f1 : {lr_grid.best_score_}')
print('best param : ', lr_grid.best_params_)

# 참고 : https://wikidocs.net/16594

best f1 : 0.9328807556080283
best param :  {'C': 0.001, 'penalty': 'l2'}


c:\Users\dbswo\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
25 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\dbswo\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\dbswo\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py", line 1091, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "c:\Users\dbswo\AppData\Local\Programs\Python\

## RandomForest

In [7]:
x_train, x_test, y_train, y_test = train_test_split(input, target, random_state=42, test_size=0.2)

rs = RobustScaler()
x_train = rs.fit_transform(x_train)
x_test = rs.fit_transform(x_test)

rf = RandomForestClassifier(random_state=42)

rf_param_grid = {'n_estimators' : [100, 200],
                'max_depth' : [3, 5, 7],
                'min_samples_leaf' : [8, 12, 16],
                'min_samples_split' : [8, 16, 20]}

rf_grid = GridSearchCV(rf, param_grid=rf_param_grid, scoring='roc_auc', verbose=0, n_jobs=1)
rf_grid.fit(x_train, y_train)

print(f'best roc : {rf_grid.best_score_}')
print('best param : ', rf_grid.best_params_)

## 참고 : https://techblog-history-younghunjo1.tistory.com/102
## https://jaaamj.tistory.com/35


best roc : 0.9254624163715072
best param :  {'max_depth': 5, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 200}


In [8]:
pred = rf_grid.best_estimator_.predict_proba(x_test)
roc = roc_auc_score(y_test, pred[:,1])
print(roc)

0.9065934065934065


In [5]:
x_train, x_test, y_train, y_test = train_test_split(input, target, random_state=42, test_size=0.2)

rf = RandomForestClassifier(random_state=42)

rf_param_grid = {'n_estimators' : [100, 200],
                'max_depth' : [3, 5, 7],
                'min_samples_leaf' : [8, 12, 16],
                'min_samples_split' : [8, 16, 20]}

rf_grid = GridSearchCV(rf, param_grid=rf_param_grid, scoring='roc_auc', verbose=0, n_jobs=1)
rf_grid.fit(x_train, y_train)

print(f'best f1 : {rf_grid.best_score_}')
print('best param : ', rf_grid.best_params_)

## 참고 : https://techblog-history-younghunjo1.tistory.com/102
## https://jaaamj.tistory.com/35

best f1 : 0.9254624163715072
best param :  {'max_depth': 5, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 200}


## SVM

In [14]:
x_train, x_test, y_train, y_test = train_test_split(input, target, random_state=42, test_size=0.2)

rs = RobustScaler()
x_train = rs.fit_transform(x_train)
x_test = rs.fit_transform(x_test)

svc = SVC(random_state=42)

svc_param_grid = {'C' : [0.001, 0.01, 0.1, 1, 10],
                'gamma' : [0.001, 0.01, 0.1, 1, 10]}

svc_grid = GridSearchCV(svc, param_grid=svc_param_grid, scoring='roc_auc', verbose=0, n_jobs=1)
svc_grid.fit(x_train, y_train)

print(f'best roc : {svc_grid.best_score_}')
print('best param : ', svc_grid.best_params_)

best roc : 0.9378591105863834
best param :  {'C': 10, 'gamma': 0.001}


In [15]:
import joblib
joblib.dump(rs, 'robustscaler.pkl')

['robustscaler.pkl']

In [10]:
pred = rf_grid.best_estimator_.predict_proba(x_test)
roc = roc_auc_score(y_test, pred[:,1])
print(roc)

0.9065934065934065


In [16]:
x_train, x_test, y_train, y_test = train_test_split(input, target, random_state=42, test_size=0.2)

svc = SVC(random_state=42)

svc_param_grid = {'C' : [0.001, 0.01, 0.1, 1, 10],
                'gamma' : [0.001, 0.01, 0.1, 1, 10]}

svc_grid = GridSearchCV(svc, param_grid=svc_param_grid, scoring='roc_auc', verbose=0, n_jobs=1)
svc_grid.fit(x_train, y_train)

print(f'best f1 : {svc_grid.best_score_}')
print('best param : ', svc_grid.best_params_)

best f1 : 0.8655745769382133
best param :  {'C': 1, 'gamma': 0.001}


### CatBoostClassifier는 파라미터 조정이 성능에 크게 영향을 미치지 않는다는 말이 많아 일단 생략함
##### https://velog.io/@jus6886/Catboost
##### https://undeadkwandoll.tistory.com/61
##### https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART002698429
#### CatBoost 설명
##### https://dailyheumsi.tistory.com/136
##### https://techblog-history-younghunjo1.tistory.com/199
##### https://heeya-stupidbutstudying.tistory.com/43?category=950711


In [37]:
x_train, x_test, y_train, y_test = train_test_split(input, target, random_state=42, test_size=0.2)

rs = RobustScaler()
x_train = rs.fit_transform(x_train)
x_test = rs.fit_transform(x_test)

cat = CatBoostClassifier(random_state=42, verbose=0)
cat.fit(x_train, y_train)
pred = cat.predict(x_test)

acc = accuracy_score(y_test, pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
roc = roc_auc_score(y_test, pred)

print(roc)

0.8145604395604396
